In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
MERCK_FILE= '../Datasets/MerckActivity/TrainingSet/ACT2_competition_training.csv'
with open(MERCK_FILE) as f:
    cols = f.readline().rstrip('\n').split(',') # Read the header line and get list of column names
    # Load the actual data, ignoring first column and using second column as targets.
    X = np.loadtxt(MERCK_FILE, delimiter=',', usecols=range(2, len(cols)), skiprows=1, dtype=np.uint8) 
    y = np.loadtxt(MERCK_FILE, delimiter=',', usecols=[1], skiprows=1)

In [3]:
MERCK_FILE2= '../Datasets/MerckActivity/TrainingSet/ACT4_competition_training.csv'
with open(MERCK_FILE2) as f:
    cols = f.readline().rstrip('\n').split(',') # Read the header line and get list of column names
    # Load the actual data, ignoring first column and using second column as targets.
    X_ACT4 = np.loadtxt(MERCK_FILE2, delimiter=',', usecols=range(2, len(cols)), skiprows=1, dtype=np.uint8) 
    y_ACT4 = np.loadtxt(MERCK_FILE2, delimiter=',', usecols=[1], skiprows=1)

In [4]:
#saving into files ACT2
from tempfile import TemporaryFile
outfileACT2 = TemporaryFile()
np.savez(outfileACT2, x_ACT2 = X, y_ACT2 = y)
_ = outfileACT2.seek(0) 

#saving into files ACT4
outfileACT4 = TemporaryFile()
np.savez(outfileACT4, x_ACT4 = X_ACT4, y_ACT4 = y_ACT4)
_ = outfileACT4.seek(0) 

In [5]:
# loading file ACT2
npzfile_ACT2 = np.load(outfileACT2, allow_pickle=True)
print(npzfile_ACT2.files)

# loading file ACT2
npzfile_ACT4 = np.load(outfileACT4, allow_pickle=True)
print(npzfile_ACT4.files)

['x_ACT2', 'y_ACT2']
['x_ACT4', 'y_ACT4']


In [6]:
# Dataframe
X_ACT2 = pd.DataFrame(npzfile_ACT2['x_ACT2'])
y_ACT2 = pd.DataFrame(npzfile_ACT2['y_ACT2'])

X_ACT4 = pd.DataFrame(npzfile_ACT4['x_ACT4'])
y_ACT4 = pd.DataFrame(npzfile_ACT4['y_ACT4'])

X_ACT4.head()


,0,1,2,3,4,5,6,7,8,9,...,4296,4297,4298,4299,4300,4301,4302,4303,4304,4305
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
class RegressionModels():


  def scores(self,y_test,y_pred, model):
    print("Variance Score : " , explained_variance_score(y_test, y_pred))
    print("R2 Score : " ,r2_score(y_test,y_pred))
    print("Root Mean Square : ",math.sqrt(mean_squared_error(y_test, y_pred)))
    print("Best Parameters : ", model.best_params_ )

  def svmRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------SVM Regression Starts------------\n\n ")
    clf = SVR()
    svm_grid = GridSearchCV(clf, params, verbose=False, cv=3,return_train_score=True)
    svm_grid.fit(X_train,y_train)
    svm_predict = svm_grid.predict(X_test)
    print("Best Parameters : ", svm_grid.best_params_ )
    self.scores(Y_test, svm_predict, svm_grid)
    print("\n-----------SVM Regression ends------------\n\n ")

  def decisionTreeRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Decision Tree Regression Starts------------\n\n")
    DTregressor = DecisionTreeRegressor(random_state=0)
    DT_grid = GridSearchCV(DTregressor, params, verbose=False, cv=3,return_train_score=True)
    DT_grid.fit(X_train,y_train)
    DT_predict = DT_grid.predict(X_test)
    self.scores(Y_test, DT_predict, DT_grid)
    print("\n-----------Decision Tree Regression Ends------------\n\n")

  def randomForestRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Random Forest Regression Starts------------\n\n")
    RFRegressor = RandomForestRegressor(random_state=0)
    RF_grid = GridSearchCV(RFRegressor, params, verbose=False, cv=3, return_train_score=True)
    RF_grid.fit(X_train,y_train)
    RF_predict = RF_grid.predict(X_test)
    self.scores(Y_test, RF_predict, RF_grid)
    print("\n-----------Random Forest Regression Ends------------\n\n")
  
  def adaBoostRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------AdaBoost Regression Starts------------\n\n")
    AdaRegressor = AdaBoostRegressor(random_state=0)
    adaBoost_grid = GridSearchCV(AdaRegressor, params, verbose=False, cv=3,return_train_score=True)
    adaBoost_grid.fit(X_train,y_train)
    adaBoost_predict = adaBoost_grid.predict(X_test)
    self.scores(Y_test, adaBoost_predict, adaBoost_grid)
    print("\n-----------AdaBoost Regression Ends------------\n\n")

  def gaussianProcessRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------GaussianProcess Regression Starts------------\n\n")
    GPRRegressor = GaussianProcessRegressor(random_state=0)
    GPR_grid = GridSearchCV(GPRRegressor, params, verbose=False, cv=3,return_train_score=True)
    GPR_grid.fit(X_train,y_train)
    GPR_predict = GPR_grid.predict(X_test)
    self.scores(Y_test, GPR_predict, GPR_grid)
    print("\n-----------GaussianProcess Regression Ends------------\n\n")

  def LinearRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Linear Regression Starts------------\n\n")
    LinearRegressor = LinearRegression()
    linearRegression_grid = GridSearchCV(LinearRegressor, params, verbose=False, cv=3,return_train_score=True)
    linearRegression_grid.fit(X_train,y_train)
    linearRegression_predict = linearRegression_grid.predict(X_test)
    self.scores(Y_test, linearRegression_predict, linearRegression_grid)
    print("\n-----------Linear Regression Ends------------\n\n")

  def mlpRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Neural Network Regression Starts------------\n\n")
    MLPRegressor_obj = MLPRegressor(random_state=0)
    MLPRegressor_grid = GridSearchCV(MLPRegressor_obj, params, verbose=False, cv=3,return_train_score=True)
    MLPRegressor_grid.fit(X_train,y_train)
    MLPRegressor_predict = MLPRegressor_grid.predict(X_test)
    self.scores(Y_test, MLPRegressor_predict, MLPRegressor_grid)
    print("\n-----------Neural Network Regression Ends------------\n\n")
    

  def train_split(self,X,y,test_size=0.2,random_state=0):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    return X_train,X_test,y_train,y_test
 
  def train_all__models(self, X, y):
    svm_regression_params =  { 'C' : np.logspace(0, 3, 4), 'gamma' : np.logspace(-2, 1, 4)}
    dt_params = {'max_depth' : np.arange(1, 10, 10),'min_samples_split': np.arange(0.1, 1.0, 10)}
    rd_params = {'n_estimators' : np.arange(10,100,10),'max_depth' : np.arange(1,6,2)}
    ada_params = {'n_estimators' : np.arange(10,100,10)}
    gpr_params = {'n_restarts_optimizer' : np.arange(1,10,1)}
    linear_params = {'n_jobs' : np.arange(1,5,1)}
    mlp_params = {'hidden_layer_sizes': np.arange(30,150,20),'learning_rate': ['constant','invscaling','adaptive'],'max_iter': np.arange(20,200,50)}
    X_train,X_test,y_train,y_test = self.train_split(X,y)
    self.svmRegression(X_train,X_test,y_train,y_test, svm_regression_params)
    self.decisionTreeRegression(X_train,X_test,y_train,y_test, dt_params)
    self.randomForestRegression(X_train,X_test,y_train,y_test, rd_params)
    self.adaBoostRegression(X_train,X_test,y_train,y_test, ada_params)
    self.gaussianProcessRegression(X_train,X_test,y_train,y_test, gpr_params)
    self.LinearRegression(X_train,X_test,y_train,y_test, linear_params)
    self.mlpRegression(X_train,X_test,y_train,y_test, mlp_params)

  def wine_quality(self):
    df = pd.read_csv('../Datasets/Wine Quality/winequality-red.csv',delimiter=';')
    df.dropna(axis=0,inplace=True)
    #print(df.corr()['quality'].drop('quality'))
    X = df[df.columns[0:11]]
    y = df[df.columns[11:12]]
    self.train_all__models(X, y.values.ravel())

  def communities(self):
    columns_data = ['state','county','community','communityname','fold','population','householdsize','racepctblack','racePctWhite','racePctAsian','racePctHisp','agePct12t21',
                    'agePct12t29','agePct16t24','agePct65up','numbUrban','pctUrban','medIncome','pctWWage','pctWFarmSelf','pctWInvInc','pctWSocSec','pctWPubAsst','pctWRetire','medFamInc',
                    'perCapInc','whitePerCap','blackPerCap','indianPerCap','AsianPerCap','OtherPerCap','HispPerCap','NumUnderPov','PctPopUnderPov','PctLess9thGrade',
                    'PctNotHSGrad','PctBSorMore','PctUnemployed','PctEmploy','PctEmplManu','PctEmplProfServ','PctOccupManu','PctOccupMgmtProf','MalePctDivorce','MalePctNevMarr',
                    'FemalePctDiv','TotalPctDiv','PersPerFam','PctFam2Par','PctKids2Par','PctYoungKids2Par','PctTeen2Par','PctWorkMomYoungKids','PctWorkMom','NumIlleg','PctIlleg',
                    'NumImmig','PctImmigRecent','PctImmigRec5','PctImmigRec8','PctImmigRec10','PctRecentImmig','PctRecImmig5','PctRecImmig8','PctRecImmig10','PctSpeakEnglOnly','PctNotSpeakEnglWell'
                    ,'PctLargHouseFam','PctLargHouseOccup','PersPerOccupHous','PersPerOwnOccHous','PersPerRentOccHous','PctPersOwnOccup','PctPersDenseHous','PctHousLess3BR',
                    'MedNumBR','HousVacant','PctHousOccup','PctHousOwnOcc','PctVacantBoarded','PctVacMore6Mos','MedYrHousBuilt','PctHousNoPhone','PctWOFullPlumb','OwnOccLowQuart','OwnOccMedVal',
                    'OwnOccHiQuart','RentLowQ','RentMedian','RentHighQ','MedRent','MedRentPctHousInc','MedOwnCostPctInc','MedOwnCostPctIncNoMtg','NumInShelters','NumStreet','PctForeignBorn',
                    'PctBornSameState','PctSameHouse85','PctSameCity85','PctSameState85','LemasSwornFT','LemasSwFTPerPop','LemasSwFTFieldOps','LemasSwFTFieldPerPop','LemasTotalReq','LemasTotReqPerPop',
                    'PolicReqPerOffic','PolicPerPop','RacialMatchCommPol','PctPolicWhite','PctPolicBlack','PctPolicHisp','PctPolicAsian','PctPolicMinor','OfficAssgnDrugUnits','NumKindsDrugsSeiz',
                    'PolicAveOTWorked','LandArea','PopDens','PctUsePubTrans','PolicCars','PolicOperBudg','LemasPctPolicOnPatr','LemasGangUnitDeploy','LemasPctOfficDrugUn','PolicBudgPerPop',
                    'ViolentCrimesPerPop'
                    ]
    df = pd.read_csv('communities.data',delimiter=',',names=columns_data)

    #print("Before Removing the Non Predictable Features \n")
    #print("Shape Before Removing :" +str(df.shape) + "\n")
    #print(df.head())
    df = df.replace('?',np.nan)
    #print("After Removing the Non predictable Features \n ")
    #print("Shape After Removing :" + str(df.shape) + "\n")
    #print(df.head())
    print("Before Droping data Shape " + str(df.shape))
    #According to Dataset Description there are 5 non predictive features which can be removed
    df = df.drop(['fold','community','state','communityname','county'],axis=1)

    print("Checking the Columns Containing the null Values")
    
    #for i in range(0,120,41):
    #  print(df.iloc[:,i:i+41].isna().sum())
    #  print("\n")
    median_value = df.iloc[:,25].median(skipna = True)
    df.iloc[130,25] = median_value
    
    df = df.dropna(axis=1)
    print(df.columns)
    print("After Droping data Shape " + str(df.shape))
    #Replacing the columns with median
  

    print("Number of Missing Values in column is " + str(df.iloc[:,25].isna().sum()))
    print(df.shape)
    X = df[df.columns[0:100]]
    y = df[df.columns[100:101]]
    print(X.describe())
    #print(y)
    #print(X.head())
    self.train_all__models(X, y.values.ravel())


In [8]:
regressionModels = RegressionModels()
regressionModels.train_all__models(X_ACT4, y_ACT4)

-----------SVM Regression Starts------------

 


/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d

/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d

Best Parameters :  {'C': 100.0, 'gamma': 0.01}
Variance Score :  0.23245696807916194
R2 Score :  0.2324407900070704
Root Mean Square :  0.4951064581471691
Best Parameters :  {'C': 100.0, 'gamma': 0.01}

-----------SVM Regression ends------------

 
-----------Decision Tree Regression Starts------------


Variance Score :  0.23065159398749846
R2 Score :  0.22965623623974596
Root Mean Square :  0.4960037195115153
Best Parameters :  {'max_depth': 1, 'min_samples_split': 0.1}

-----------Decision Tree Regression Ends------------


-----------Random Forest Regression Starts------------




/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selec

...

/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was 

/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (120) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was

/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Variance Score :  0.1695664549375021
R2 Score :  0.164810228382406
Root Mean Square :  0.516458264114007
Best Parameters :  {'hidden_layer_sizes': 110, 'learning_rate': 'constant', 'max_iter': 70}

-----------Neural Network Regression Ends------------




/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
